## Day 46 Lecture 1 Assignment

In this assignment, we will calculate a distance matrix for geographical Starbucks data and use it to identify locations that are close together and far apart. We will perform clustering on this dataset later on.

We will be using the "haversine" package to compute geographical distance. It can be pip installed.

In [1]:
!pip install haversine

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from haversine import haversine

In [4]:
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/starbucks_locations.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25600 entries, 0 to 25599
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           25600 non-null  object 
 1   Store Number    25600 non-null  object 
 2   Store Name      25600 non-null  object 
 3   Ownership Type  25600 non-null  object 
 4   Street Address  25598 non-null  object 
 5   City            25585 non-null  object 
 6   State/Province  25600 non-null  object 
 7   Country         25600 non-null  object 
 8   Postcode        24078 non-null  object 
 9   Phone Number    18739 non-null  object 
 10  Timezone        25600 non-null  object 
 11  Longitude       25599 non-null  float64
 12  Latitude        25599 non-null  float64
dtypes: float64(2), object(11)
memory usage: 2.5+ MB


In [16]:
df.head(3)

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39


Below is a convenience function for calculating geographical distance matrices using lat-long data.

In [17]:
def geo_sim_matrix(df, col_name = 'Coordinates'):
    """
    A function that computes a geographical distance matrix (in miles).
    Each row in the dataframe should correspond to one location.
    In addition, the dataframe must have a column containing the lat-long of each location as a tuple (i.e. (lat, long)).
    
    Parameters:
        df (pandas dataframe): an nxm dataframe containing the locations to compute similarities between.
        col (string): the name of the column containing the lat-long tuples.
        
    Returns:
        distance (pandas dataframe): an nxn distance matrix between the geographical coordinates of each location.
    """
    
    df = df.copy()
    df.reset_index(inplace=True)
    haver_vec = np.vectorize(haversine, otypes=[np.float32])
    distance = df.groupby('index').apply(lambda x: pd.Series(haver_vec(df[col_name], x[col_name])))
    distance = distance / 1.609344  # converts to miles
    distance.columns = distance.index
    
    return distance


This dataset contains the latitude and longitude (as well as several other details we will not be using) of every Starbucks in the world as of February 2017. Each row consists of the following features, which are generally self-explanatory:

- Brand
- Store Number
- Store Name
- Ownership Type
- Street Address
- City
- State/Province
- Country
- Postcode
- Phone Number
- Timezone
- Longitude
- Latitude

Load in the dataset.

In [ ]:
# answer goes here


Begin by narrowing down the dataset to a specific geographic area of interest. Since we will need to manually compute a distance matrix, which will be on the order of $n^{2}$ in terms of size, we would recommend choosing an area with 3000 or less locations. In this example, we will use Hawaii, which has about 100 locations; for reference, California has about 2800 locations. Feel free to choose a different region that is of more interest to you, if desired.

Subset the dataframe to only include records for Starbucks locations in Hawaii.

In [109]:
# answer goes here
dfhi = df.loc[df['State/Province']=='HI', :]
dfhi.head(2)


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
17202,Starbucks,21034-73360,Aiea Shopping Center,Company Owned,99-115 Aiea Heights Drive #125,Aiea,HI,US,967013913,808-484-1488,GMT-10:00 Pacific/Honolulu,-157.93,21.38
17203,Starbucks,21053-99755,Stadium Marketplace,Company Owned,4561 Salt Lake Boulevard,Aiea,HI,US,968183167,808-488-9313,GMT-10:00 Pacific/Honolulu,-157.93,21.37


In [110]:
df.head(2)


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42


In [111]:
dfhi['Coordinates'] = dfhi[['Latitude', 'Longitude']].apply(tuple, axis=1)
dfhi.head()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates
17202,Starbucks,21034-73360,Aiea Shopping Center,Company Owned,99-115 Aiea Heights Drive #125,Aiea,HI,US,967013913,808-484-1488,GMT-10:00 Pacific/Honolulu,-157.93,21.38,"(21.38, -157.93)"
17203,Starbucks,21053-99755,Stadium Marketplace,Company Owned,4561 Salt Lake Boulevard,Aiea,HI,US,968183167,808-488-9313,GMT-10:00 Pacific/Honolulu,-157.93,21.37,"(21.37, -157.93)"
17204,Starbucks,21006-10033,Kaonohi St & Kam Hwy - Pearlridge,Company Owned,98-125 Kaonohi Street,Aiea,HI,US,967012318,808-484-9548,GMT-10:00 Pacific/Honolulu,-157.94,21.38,"(21.38, -157.94)"
17205,Starbucks,21005-10034,Pearlridge Mall Uptown,Company Owned,98-1005 Moanalua Road,Aiea,HI,US,967014705,808-484-9355,GMT-10:00 Pacific/Honolulu,-157.94,21.39,"(21.39, -157.94)"
17206,Starbucks,21063-101700,Waimalu Shopping Center,Company Owned,"98-1277 Kaahumanu Street, Building E, Unit 7, ...",Aiea,HI,US,967015314,808-484-5802,GMT-10:00 Pacific/Honolulu,-157.95,21.39,"(21.39, -157.95)"


The haversine package takes tuples with 2 numeric elements and interprets them as lat-long to calculate distance, so add a new column called "Coordinates" that converts the lat and long in each row into a tuple. In other words, the last two columns of the dataframe should initially look like this:

**Latitude, Longitude**  
-121.64, 39.14  
-116.40, 34.13  
...

After adding the new column, the last three columns should look like this:

**Latitude, Longitude, Coordinates**  
-121.64, 39.14, (-121.64, 39.14)  
-116.40, 34.13, (-116.40, 34.13)  
...

In [127]:
# answer goes here
dfdist = geo_sim_matrix(dfhi, col_name = 'Coordinates')

dfdist.head(2)


index,17202,17203,17204,17205,17206,17207,17208,17209,17210,17211,17212,17213,17214,17215,17216,17217,17218,17219,17220,17221,17222,17223,17224,17225,17226,17227,17228,17229,17230,17231,17232,17233,17234,17235,17236,17237,17238,17239,17240,17241,...,17261,17262,17263,17264,17265,17266,17267,17268,17269,17270,17271,17272,17273,17274,17275,17276,17277,17278,17279,17280,17281,17282,17283,17284,17285,17286,17287,17288,17289,17290,17291,17292,17293,17294,17295,17296,17297,17298,17299,17300
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
17202,0.000000,0.690934,0.643386,0.944091,1.460500,6.743576,6.743576,218.428635,218.976944,217.137772,218.428635,9.442554,7.130735,8.498165,3.514091,2.837658,9.442554,10.386974,3.514091,8.498165,8.981524,8.006303,10.362812,2.837658,9.016180,3.514091,3.514091,8.073252,11.871831,4.748090,8.498165,10.362812,1.381868,7.130735,11.354635,3.514091,8.073252,15.755173,6.609475,6.609475,...,101.817009,12.243432,12.243432,173.338730,173.338730,173.818802,173.679764,8.476842,8.199399,8.807722,100.835060,10.860067,10.860067,10.039995,104.882942,104.882942,105.469711,104.163841,172.870682,87.654320,87.654320,87.381813,85.394745,101.634392,100.154015,101.634392,8.004014,7.062008,7.677204,2.920923,109.065346,12.792754,2.373938,16.607107,166.981064,107.264542,98.783943,5.329021,5.675488,6.470637
17203,0.690934,0.000000,0.944121,1.524305,1.888212,6.417289,6.417289,218.067474,218.616730,216.778824,218.067474,8.949539,6.609684,8.006588,2.837667,2.170391,8.949539,9.892798,2.837667,8.006588,8.573816,7.546013,9.915722,2.170391,8.498434,2.837667,2.837667,7.554044,11.483742,4.382197,8.006588,9.915722,0.690934,6.609684,10.948190,2.837667,7.554044,15.465081,6.122129,6.122129,...,101.592422,12.302195,12.302195,172.866028,172.866028,173.344666,173.311035,8.616761,8.457584,9.048605,101.161568,10.660760,10.660760,9.872494,104.601028,104.601028,105.180283,104.398209,172.396683,87.386826,87.386826,87.118973,85.142502,101.916420,100.444832,101.916420,8.496008,7.552127,8.246893,3.304420,108.834190,13.215210,2.050284,16.793444,166.565231,106.962166,98.552231,5.548625,6.120771,6.580589


Calculate the distance matrix using the starter code/function geo_sim_matrix() provided earlier in the notebook. It assumes the column containing the coordinates for each location is called "Coordinates". Examine the docstring for more details.

Note: the latitude and longitudes provided only go out to two decimal places, which limits the resolution of the distance calculations to about 0.5 miles. Distances that are very small may not be accurately represented here (e.g. several instances of "0 distance" for distinct Starbucks locations in very close proximity).

In [128]:
# answer goes here




For each Starbucks, identify its nearest neighboring location in Hawaii (and presumably in the world). Save the output to a dataframe with three columns: Location, Nearest Neighbor, and Distance (Miles).

In [129]:
dfdist.columns[np.argmin(dfdist.drop(17202, axis = 1).loc[17202, :])]
dfdist.columns[1]

17203

In [149]:
# answer goes here
minimum = []
nearestneighbor = []
for x in dfdist.index:
  minimum.append(dfdist.drop(x, axis = 1).loc[x,:].min())
  nearestneighbor.append(dfdist.columns[np.argmin(dfdist.drop(x, axis = 1).loc[x, :])])

dfsb = pd.DataFrame()  
dfsb['location'] = dfdist.index
dfsb['distance'] = minimum
dfsb['nearest neighbor'] = nearestneighbor
dfsb
print(dfsb.head(2))
dfhi.head(2)


   location  distance  nearest neighbor
0     17202  0.643386             17203
1     17203  0.690934             17202


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates,location
17202,Starbucks,21034-73360,Aiea Shopping Center,Company Owned,99-115 Aiea Heights Drive #125,Aiea,HI,US,967013913,808-484-1488,GMT-10:00 Pacific/Honolulu,-157.93,21.38,"(21.38, -157.93)",17202
17203,Starbucks,21053-99755,Stadium Marketplace,Company Owned,4561 Salt Lake Boulevard,Aiea,HI,US,968183167,808-488-9313,GMT-10:00 Pacific/Honolulu,-157.93,21.37,"(21.37, -157.93)",17203


If the nearest neighbor of a Starbucks location is far away, we could consider that Starbucks to be "on an island". Which five Starbucks in Hawaii are the most "on an island"?

In [153]:
# answer goes here
#dfsb.sort_values(by= 'distance', ascending= False)[['location', 'distance']].head(5)
#dfsb.reset_index('index')
dfhi['location']= dfhi.index
dfhi.head(2)
dfmerge = pd.merge(dfsb,dfhi, on = 'location')
dfmerge.sort_values(by = 'distance', ascending = False).head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,location,distance,nearest neighbor,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates
65,17267,15.612447,17294,Starbucks,21030-58899,Parker Ranch Center,Company Owned,67-1185 Mamalahoa Highway D108,Kamuela,HI,US,967438427,808-887-6409,GMT-10:00 Pacific/Honolulu,-155.67,20.02,"(20.02, -155.67)"
93,17295,15.612447,17267,Starbucks,21070-104953,Queens Marketplace,Company Owned,"69-201 Waikoloa Beach Drive, #1001 K-1",Waikoloa,HI,US,967385810,808-886-1888,GMT-10:00 Pacific/Honolulu,-155.88,19.91,"(19.91, -155.88)"
76,17278,8.059888,17283,Starbucks,21071-107523,Poipu Shopping Village,Company Owned,"2360 Kiahuna Plantation Drive, Suites E-70 & E-80",Koloa,HI,US,967569552,(808) 742-5144,GMT-10:00 Pacific/Honolulu,-159.46,21.88,"(21.88, -159.46)"
89,17291,7.322419,17261,Starbucks,21040-85520,Pukalani Terrace Center,Company Owned,55 Pukalani Street,Pukalani,HI,US,96768,808-573-5458,GMT-10:00 Pacific/Honolulu,-156.34,20.84,"(20.84, -156.34)"
92,17294,6.739833,17292,Starbucks,21073-102259,Waianae Mall - Farrington Hwy,Company Owned,"86-120 Farrington Highway, Waikoloa Beach Resort",Waianae,HI,US,967923000,(808) 697-8279,GMT-10:00 Pacific/Honolulu,-158.18,21.44,"(21.44, -158.18)"
